In [ ]:
%pylab inline
plt.ion()

In [ ]:
from lsst.eo_utils.base.config_utils import get_config_values
from lsst.eo_utils.bias.analysis import BiasAnalysisBySlot, make_bias_v_row_slot

In [ ]:
# This is the list of arguments to get from the default configuration
argnames = ['run', 'rafts', 'slots', 'bias', 'mask', 'butler_repo', 'outdir', 'nfiles']

# Here we are getting the default values, and updating run, butler_repo and bias
arg_dict = get_config_values(argnames, run='6106D', butler_repo='TS8', bias='spline', nfiles=2, plot=True)

# Lets only analyze the darks
arg_dict['acq_types'] = ['DARK']

In [ ]:
# The run number is special, so we pull that out
#run_num = arg_dict.pop('run')

In [ ]:
print(arg_dict)

In [ ]:
# make_bias_v_row_slot is a function that look like:
# def make_bias_v_row_slot(butler, slot_data, **kwargs)
#
# Passing this function to a BiasAnalysisBySlot object will
# connect it to the function that loop over slots and 
# get the bias files for each for for the given run
driver = BiasAnalysisBySlot(make_bias_v_row_slot, argnames)

In [ ]:
# Use the driver to run over all the slots,
driver.run(interactive=True, **arg_dict)

In [ ]:
# That last line is equivalent to the code below

# Get the butler
butler = driver.get_butler(arg_dict['butler_repo'])

# Get the run number
run_num = arg_dict['run']

# We will need the raft name
htype, raft = driver.get_hardware(butler, run_num)
# This is a dictionary of dataIds, sorted by slot and imageType
data = driver.get_data(butler, run_num, **arg_dict)[raft]
# Loop over the slots and run the analysis function
for slot, slotData in data.items():
    make_bias_v_row_slot(butler, slotData, run_num=run_num, slot=slot, raft=raft, **arg_dict)